# PS1: Your first library-free neural network!  

Advanced Learning 2025


For SUBMISSION:   

Please upload the complete and executed `ipynb` to your git repository. Verify that all of your output can be viewed directly from github, and provide a link to that git file below.

~~~
STUDENT ID: 345801963
~~~

~~~
STUDENT GIT LINK: https://github.com/DanAbergel/Adv.-computational-learning-and-data-analysis/tree/main/HW1/PS1_Part2_Hello_NN_2025_ID_345801963.ipynb
~~~
In Addition, don't forget to add your ID to the files:    
  
`PS1_Part2_HelloNN_2025_ID_[000000000].html`   


In [1]:
import numpy as np # You are allowed to nuse  only numpy.
import time


**Welcome**.   

In this part of the problem set you are set to build a complete and flexible neural network.  
This neural network will be library free (in the sense that we won't use PyTorch/Tensorflow/etc.).   

Let's do a quick review of the basic neural-network components:  


*   *Layer* - can be fully connected (dense/hidden), convolution, etc.
  * Forward propagation- the layer outputs the next layer's input
  * Backward propagation- the layer also outputs the gradient descent update
*   *Activation* Layer (e.g. ReLU) - there are no parameters, only gradients with respect to the input. We want to compute both the gradient w.r.t the parameters of the layer and to create the gradient with respect to the layer's inputs
   * *Forward propagation*- the layer outputs the next layer's input
   * *Backward propagation*- the layer also outputs the gradient descent update
*   *Loss Function* : how our model  quantifies the difference between the predicted outputs the actual (target) values  
*   *Network Wrapper*-  wraps our components together as a trainable model.






Useful resource:  
* Gradient descent for neural networks [cheat sheet](https://moodle4.cs.huji.ac.il/hu23/mod/resource/view.php?id=402297).
* Neural network architecture [cheat sheet](https://moodle4.cs.huji.ac.il/hu23/mod/url/view.php?id=402298).

### 0. Loading data

You are going to test and evaluate your home-made network on the `mnist` dataset.   
The MNIST dataset is a large dataset of handwritten digits that is commonly used for training various image and vision models.

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
# load MNIST from server
# Using a standard library (keras.datasets) to load the mnist data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(f"x_train = {x_train.shape} , y_train = {y_train.shape} , x_test = {x_test.shape} , y_test = {y_test.shape}")

x_train = (60000, 28, 28) , y_train = (60000,) , x_test = (10000, 28, 28) , y_test = (10000,)


In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
# load MNIST from server
# Using a standard library (keras.datasets) to load the mnist data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_test.shape

(10000,)

#### Data transformations





In [4]:
# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# One-hot encoding of the output.
# Currently a number in range [0,9]; Change into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = to_categorical(y_train)
# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = to_categorical(y_test)

### 1. Network's Components

Please fill-in the missing code in the code boxes below (only where  `#### SOLUTION REQUIRED ####` is specified).   

In [5]:

# This class is a general layer primitive, defining that each instance must
# have an (input,output) parameters, and 2 functions: forward+backward propogation
class Layer_Primitive:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

#### Fully Connected Layer

A fully-connected layer (a.k.a. affine, dense,linear layer) connects every input neuron to every output neuron.   
It has 2 parameters: (input, output).   
You need to define (code) the following:
* its initialization weights with random weights.
* the forward propogation calculation (as shown in class).
* the backward propogation gradients calculation (given output, as shown in class).

Parameters must be intitialized with some values. There are many ways to initialize the weights, and you are encouraged to do a quick research about the common methods. Any commonly used method will be accepted.  

1.1 (20 pts)

In [7]:
#### SOLUTION REQUIRED ####


# inherit from base class Layer
class Affine_Layer(Layer_Primitive):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        super().__init__()
        # weights dimension = output_size X input_size
        self.weights = np.random.randn(output_size,input_size)  # FILL IN THE MISSING CODE: randomly initialize the weights of the input_size, output_size size layer.
        self.bias = np.random.randn(output_size) # FILL IN THE MISSING CODE: randomly initialize the bias of the  layer.


    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data.reshape(-1)
        self.output = self.weights @ self.input.T + self.bias
        return self.output



    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_grad, learning_rate):
        # (input_size,output_size) @ output_size
        input_error = self.weights.T @ output_grad  # FILL IN THE MISSING CODE
        # (output_size , 1)@(1,input_size) 
        weights_error = output_grad.reshape(-1,1) @ self.input.reshape(1,-1)  # FILL IN THE MISSING CODE

        # update parameters
        self.weights -= learning_rate * weights_error # FILL IN THE MISSING CODE
        self.bias -= learning_rate * output_grad # FILL IN THE MISSING CODE

        return input_error

#### Activation layers

Activation functions are often a non-linear functions that aid in how well the network model adapts to and learns  the training dataset. The choice of activation function in the output layer will define the type of predictions the model can make.  



In [8]:
# inherit from base class Layer
class ActivationLayer(Layer_Primitive):
    def __init__(self, activation, activation_grad):
        self.activation = activation
        self.activation_grad = activation_grad

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_grad=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_grad, learning_rate):
        return self.activation_grad(self.input) * output_grad



You need to define (code) the following via different functions:
* the forward propogation calculation (as shown in class).
* the backward propogation gradients calculation (given output, as shown in class).

1.2 (20 pts)

In [9]:
#### SOLUTION REQUIRED ####

# activation functions and their derivatives:

def tanh(x):
    return np.tanh(x)  # FILL IN THE MISSING CODE

def tanh_grad(x):
    return 1 - tanh(x)**2 # FILL IN THE MISSING CODE

def relu(x):
    return np.maximum(0,x) # FILL IN THE MISSING CODE

def relu_grad(x):
    return np.where(x>0,1,0)# FILL IN THE MISSING CODE

def sigmoid(x):
    # FILL IN THE MISSING CODE
    return 1/(1+np.exp(-x)) # FILL IN THE MISSING CODE

def sigmoid_grad(x):
    # FILL IN THE MISSING CODE
    return sigmoid(x) * (1 - sigmoid(x))# FILL IN THE MISSING CODE

#### Loss function

1.3 (10 pts)

In [10]:
#### SOLUTION REQUIRED ####

# loss function and its derivative

def mse(y_true, y_pred):
    return np.mean((y_pred - y_true)**2) # FILL IN THE MISSING CODE

def mse_grad(y_true, y_pred):
    return 2 * (y_pred - y_true)/y_true.size # FILL IN THE MISSING CODE



#### Putting everything together

1.4 (10 pts)

In [11]:
#### SOLUTION REQUIRED (in `predict`) ####

class MyNetwork:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_grad = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use_loss(self, loss, loss_grad):
        self.loss = loss
        self.loss_grad = loss_grad


    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                grad = self.loss_grad(y_train[j], output)
                for layer in reversed(self.layers):
                    grad = layer.backward_propagation(grad, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('Training epoch %d/%d   error=%f' % (i+1, epochs, err))


    # predict output for given input
    def predict(self, x_test,y_test=np.array([])):
        if y_test.size:
           assert len(x_test)==len(y_test) # if Y is given
        # sample dimension first
        samples = len(x_test)
        result = []
        loss = 0
        correct = 0
        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = x_test[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
            # ONLY IF LABELS ARE GIVEN (Y):
            if y_test.size:
                # Evaluate the output against Y,
                # calculate loss against Y, add to `loss`:
                loss += self.loss(y_test[i],output)# FILL IN THE MISSING CODE
                target = y_test[i]
                # Evaluate the label of the output against real, and if identical,
                # add +1 to `correct`:
                if np.argmax(target) == np.argmax(output):# FILL IN THE MISSING CODE
                   correct += 1
        if y_test.size:
            mean_loss = loss/samples

            print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.
                  format(mean_loss, correct, samples,100. * correct / samples))

        return result


## 2. Testing Your Neural Network

### Defining our main neural network architecture

Define your network's architecture:  
(Please rationalize your choice of activation funciton.)
* first affine layer that takes your input and outputs 128 nodes
* `tanh/relu/sigmoid` activation layer following the first affine layer
* second affine layer that takes the first layer's input and outputs 64 nodes
* `tanh/relu/sigmoid` activation layer following the second affine layer
* third affine layer that takes your second layer's input and outputs nodes in the size of the Y labels.
* `tanh/relu/sigmoid` activation layer following the last affine layer


2.1 (5 pts)

In [12]:
#### SOLUTION REQUIRED (in `predict`) ####

# Network Architecture
net = MyNetwork()
net.add(Affine_Layer(784,128))
net.add(ActivationLayer(relu,relu_grad))
net.add(Affine_Layer(128,64))
net.add(ActivationLayer(sigmoid,sigmoid_grad))
net.add(Affine_Layer(64,y_train[0].shape[0]))
net.add(ActivationLayer(tanh,tanh_grad))


### Training!

In [13]:

# While developing, it is recommended to train your model on a subset of the data... / or low epochs.
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use_loss(mse, mse_grad)
epoch_num = 20
lr = 0.01
t1 = time.time()
net.fit(x_train, y_train, epochs=epoch_num, learning_rate=lr)
print(f"Total process time: {round(time.time() - t1,3)}")


Training epoch 1/20   error=0.493088
Training epoch 2/20   error=0.290532


/var/folders/t8/ffgjly3d0c90vfldr57f29q00000gn/T/ipykernel_1592/1485580712.py:19: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x)) # FILL IN THE MISSING CODE


Training epoch 3/20   error=0.284574
Training epoch 4/20   error=0.281328
Training epoch 5/20   error=0.279085
Training epoch 6/20   error=0.277203
Training epoch 7/20   error=0.275888
Training epoch 8/20   error=0.274898
Training epoch 9/20   error=0.274107
Training epoch 10/20   error=0.273005
Training epoch 11/20   error=0.271772
Training epoch 12/20   error=0.271151
Training epoch 13/20   error=0.270721
Training epoch 14/20   error=0.270379
Training epoch 15/20   error=0.270099
Training epoch 16/20   error=0.269871
Training epoch 17/20   error=0.269726
Training epoch 18/20   error=0.269540
Training epoch 19/20   error=0.269405
Training epoch 20/20   error=0.269312
Total process time: 335.26


### Evaluation

Exciting! Now is the time to test your model.   

    May the gradients be always in your favor.

In [16]:
output = net.predict(x_test ,y_test )


Test set: Avg. loss: 0.2681, Accuracy: 974/10000 (10%)



## 3. Benchmarking against PyTorch

How well your model performs against a similar-architecture PyTorch model?   
It is time to find out:

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset

#### Prepare the data as tensors using PyTorch DataLoader:

In [20]:
t_train =  TensorDataset(torch.Tensor(x_train),torch.Tensor(y_train))
t_test =  TensorDataset(torch.Tensor(x_test),torch.Tensor(y_test))
train_loader = torch.utils.data.DataLoader(dataset=t_train, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=t_test, batch_size=64, shuffle=False)

Define a `PyTorchNet` class with an identical architecture you used in your home-made network.

3.1 (10 pts)

In [23]:
#### SOLUTION REQUIRED  ####

class PyTorchNet(nn.Module):
    def __init__(self):
        super(PyTorchNet, self).__init__()
        input_size = 28*28 # FILL IN THE MISSING CODE
        num_classes = 10 # FILL IN THE MISSING CODE

        self.fc1 = nn.Linear(input_size, 128) # FILL IN THE MISSING CODE
        self.activ1 = nn.ReLU() # FILL IN THE MISSING CODE
        self.fc2 = nn.Linear(128,64) # FILL IN THE MISSING CODE
        self.activ2 = nn.Sigmoid() # FILL IN THE MISSING CODE
        self.fc3 = nn.Linear(64,num_classes) # FILL IN THE MISSING CODE
        self.activ3 = nn.Tanh() # FILL IN THE MISSING CODE


    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.activ1(self.fc1(x)) # FILL IN THE MISSING CODE
        x = self.activ2(self.fc2(x)) # FILL IN THE MISSING CODE
        x = self.activ3(self.fc3(x)) # FILL IN THE MISSING CODE

        return x

In [24]:

# Train the model
num_epochs = 20
pt_learning_rate = 0.01
pt_network = PyTorchNet()
optimizer = torch.optim.Adam(pt_network.parameters(), lr=pt_learning_rate)
criterion = nn.MSELoss()

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = pt_network(images)
        loss = criterion(outputs, labels)
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # A handy printout:
        if (i + 1) % 500 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/20], Step [500/938], Loss: 0.0132
Epoch [2/20], Step [500/938], Loss: 0.0051
Epoch [3/20], Step [500/938], Loss: 0.0070
Epoch [4/20], Step [500/938], Loss: 0.0108
Epoch [5/20], Step [500/938], Loss: 0.0092
Epoch [6/20], Step [500/938], Loss: 0.0093
Epoch [7/20], Step [500/938], Loss: 0.0016
Epoch [8/20], Step [500/938], Loss: 0.0033
Epoch [9/20], Step [500/938], Loss: 0.0060
Epoch [10/20], Step [500/938], Loss: 0.0041
Epoch [11/20], Step [500/938], Loss: 0.0103
Epoch [12/20], Step [500/938], Loss: 0.0063
Epoch [13/20], Step [500/938], Loss: 0.0065
Epoch [14/20], Step [500/938], Loss: 0.0058
Epoch [15/20], Step [500/938], Loss: 0.0057
Epoch [16/20], Step [500/938], Loss: 0.0026
Epoch [17/20], Step [500/938], Loss: 0.0026
Epoch [18/20], Step [500/938], Loss: 0.0093
Epoch [19/20], Step [500/938], Loss: 0.0032
Epoch [20/20], Step [500/938], Loss: 0.0017


Evaluation:

In [25]:
pt_network.eval()
test_losses = []
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = pt_network(data)
        test_loss += criterion(output, target,)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.max(1,keepdim=True)[1]).sum()

test_loss /= len(test_loader.dataset)
test_losses.append(test_loss)
print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
  test_loss, correct, len(test_loader.dataset),
  100. * correct / len(test_loader.dataset)))


Test set: Avg. loss: 0.0001, Accuracy: 9656/10000 (97%)



3.2 (10 pts)

Time for some questions:
1. Which one of the models performed better? Why?
2. Which one of the models performed faster? Why?  
3. What would you change in your network's architecture?   
4. What would you change in your model's solution algorithm?

Write your solutions here:

Answer 1: Which model performed better? Why?

The PyTorch model performed better.
It reached around 97% accuracy, while my model stayed around 10%, which is basically random guessing.
The main reason is that PyTorch handles weight initialization, gradient computation, and numerical stability in a much more robust way. In my own implementation, some mistakes in backpropagation and/or parameter updates probably caused the model to not learn properly.

⸻

Answer 2: Which model performed faster? Why?

The PyTorch model was faster.
PyTorch uses optimized tensor operations (and can also use the GPU), while my implementation runs with pure Python loops, which is much slower. So even the same architecture runs noticeably faster in PyTorch.

⸻

Answer 3: What would you change in your network’s architecture?

I would change the activation functions and the last layer.
Instead of using sigmoid and tanh, I would switch to ReLU in the hidden layers because it reduces the risk of vanishing gradients.
Also, instead of applying tanh on the output, I would output raw logits and use a softmax inside the loss function (like CrossEntropyLoss).
So the architecture would become something like: 
784 → 128 (ReLU)
128 → 64 (ReLU)
64 → 10 (logits)

Answer 4: What would you change in the training algorithm?

I would change the optimizer and the learning rate.
Using Adam instead of simple SGD usually gives better and more stable results.
I would also make sure the data is normalized properly (which PyTorch already does).
So overall: Adam optimizer, smaller learning rate (e.g., 0.001), and CrossEntropyLoss for the output layer.



## 4. The Network Wars!

Here is your chance to play with your model's architecture in order to break your own benchmark set eariler.  
You can add/remove layers, play with their sizes, types, etc.   
You can add a new loss if you wish, or anything else that will fairly give your model an advantage over base.  

4.1 (15 pts)

In [26]:
#### SOLUTION REQUIRED  ####
class BetterNet(nn.Module):
    def __init__(self):
        super(BetterNet, self).__init__()
        
        self.layer1 = nn.Linear(28*28, 256)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)

        self.layer2 = nn.Linear(256, 128)
        self.act2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.layer3 = nn.Linear(128, 64)
        self.act3 = nn.ReLU()

        self.output = nn.Linear(64, 10)  # logits, no activation here

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.dropout1(self.act1(self.layer1(x)))
        x = self.dropout2(self.act2(self.layer2(x)))
        x = self.act3(self.layer3(x))
        x = self.output(x)
        return x